# アンサンブルカルマンフィルタによるデータ同化

このノートブックでは，01FreeRun と同様に異なる外力を与えた50メンバーのアンサンブル計算を行い，アンサンブルカルマンフィルタにより観測値を同化する。観測データは187.5秒から7.5秒おきに40グリッド中の8グリッドで与えられる。観測データは，データファイル **obs_noise.dat** から読み込む。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import advdiff

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['mathtext.fontset'] = 'cm'

----

## シミュレーション＆同化

In [ ]:
params = {
    'nx': 40,
    'dx': 10.0,
    'dt': 0.5,
    'u': 2.0,
    'nu': 5.0,
    'freq_euler': 15
}

In [ ]:
# 外力項を与える関数の設定
# 時刻を引数として呼び出すと各格子点における外力の値を返す
x = np.arange(params['nx']) * params['dx']

#
frc_x = np.sin(np.pi * x / 60.) * np.where(x < 60., 1.,  0.)
frc_t = lambda t: np.sin(2. * np.pi * t / 120.)

In [ ]:
vo = np.loadtxt('./data/state_obs.dat')

In [ ]:
nt = 1200
freq_out = 15
n_ensemble = 100

it = np.arange(nt+1, dtype=np.int64)
mask_obs = (it >= 375) & (it % 15 == 0)

In [ ]:
H = np.zeros((8,40))
for i, j in enumerate([16, 18, 20, 22, 24, 26, 28, 30]):
    H[i,j] = 1.0

In [ ]:
class EnsembleKalmanFilter:
    
    def __init__(self, ensemble, H, obs, noise):
        self.ensemble = ensemble
        self.H = H
        self.obs = obs.reshape((-1,1))
        self.noise = noise
        self.R = np.identity(len(obs)) * self.noise * self.noise
    
    def fit(self):
        P = np.cov(self.ensemble, rowvar=False, bias=False) # (40,40)
        A = np.linalg.inv(self.R + self.H @ P @ self.H.T)   # (8,8)
        K = P @ self.H.T @ A                                # (40,8)
        d = self.obs - self.H @ self.ensemble.T             # (8,nens)
        update = (K @ d).T                             # (nens,40)
        return update

In [ ]:
model = advdiff.EnsembleModel(n_ensemble, **params)

vda = [model.data]

for it in range(nt):
    _t = it * model.dt
    _ft = frc_t(_t) + np.random.normal(loc=0.0, scale=4.0, size=model.n_ensemble)
    frc = _ft.reshape((-1,1)) @ frc_x.reshape((1,-1))
    model.step_forward(force=frc)
    if mask_obs[it+1]:
        enkf = EnsembleKalmanFilter(model.data, H, vo[(it+1)//15,:], noise=4.0)
        update = enkf.fit()
        model.data = model.data + update
    if model.it % freq_out == 0:
        vda.append(model.data)

vda = np.array(vda)

In [ ]:
# 真値の読み込み
vt = np.loadtxt('./data/state_true.dat')
vs = np.loadtxt('./data/state_sim.dat')

In [ ]:
it = 70

fig, ax = plt.subplots(figsize=(8,3))
plt.plot(x, vt[it,:], '-', c='C0', lw=2, label='True')
plt.plot(x, vs[it,:], '-', c='C1', lw=2, label='Simulation')
plt.plot(x, vda[it,:,:].mean(axis=0), '-', c='C3', label='DA')
plt.plot(x @ H.T, vo[it,:], '^', c='C2', label='Observation')
#for iens in range(n_ensemble):
#    label = 'Simulation' if iens == 0 else '__nolegend__'
#    plt.plot(x, vda[it,iens,:], '-', c='C2', lw=0.5, label=label)
plt.xlim(0, 400)
plt.ylim(-40, 40)
plt.xlabel(r'$x$ [km]')
plt.ylabel(r'$C$')
plt.legend(loc='lower right', framealpha=0.7)
plt.grid(ls='--', c='0.5', lw=0.3)
plt.title(r'Free Run ($t={:4.1f}$)'.format(it * 7.5))
plt.tight_layout()

アンサンブルメンバーの数を指定

In [ ]:
n_ensemble = 50

外力の設定：外力は7.5秒おきの値を線形補間することにより求める。$i$ 番目のアンサンブルメンバーに対する時刻 $t$ における外力項は，<code>frc_t\[i\](t) * frc_x</code> により計算される（格子数と同サイズの配列）。

In [ ]:
frc_noise = np.fromfile('./data/noise/frc_noise.dat', dtype=np.float64, count=n_enesemble * ad.nt_out).reshape((n_ensemble, ad.nt_out))

x = ad.x
t = np.arange(ad.nt_out) * ad.dt_out
frc_x = np.sin(np.pi * x / 60.) * np.where(x < 60., 1.,  0.)
frc_t = np.sin(2. * np.pi * t / 120.)
frc_t = [interpolate.interp1d(t, frc_t+frc_noise[i,:], kind='linear') for i in range(n_ensemble)]

観測行列 <code>H</code>，観測誤差共分散行列 <code>R</code> の準備。

In [ ]:
H = np.zeros((ad.nx_obs, ad.nx), dtype=np.float64)
for i, j in enumerate(ad.pos_obs):
    H[i,j] = 1.0

R = np.identity(ad.nx_obs) * 8.0**2.

観測データの読み込み

In [ ]:
v_obs = np.loadtxt('./data/state_obs.dat')

出力の準備：各アンサンブルメンバーに対する計算結果は個別のファイル（./data/EnKF/state_??.dat）に出力する。

In [ ]:
fps = [open('./data/EnKF/state_%02d.dat' % (i+1), 'w') for i in range(n_ensemble)]
fmt = '%8.3f' * ad.nx

計算の実行

In [ ]:
model = ad.EnsembleSimulation(n_ensemble=n_ensemble)

for i, fp in enumerate(fps):
    print(fmt % tuple(model.data[i,:]), file=fp)

for it in range(ad.nt):
    tm = it * ad.dt
    frc = np.array([frc_t[i](tm) * frc_x for i in range(n_ensemble)])
    model.step_forward(force=frc)
    if (model.it % ad.freq_obs == 0) & (model.it * ad.dt >= 187.5):
        B = np.cov(model.data, rowvar=False, bias=False)
        A = np.linalg.inv(R + H @ B @ H.T)
        K = B @ H.T @ A
        d = v_obs[model.it//ad.freq_out,:].reshape(-1,1) - H @ model.data.T
        model.data = model.data + (K @ d).T
    if model.it % ad.freq_out == 0:
        for i, fp in enumerate(fps):
            print(fmt % tuple(model.data[i,:]), file=fp) 

for fp in fps:
    fp.close()

## 結果のプロット

In [ ]:
v_sim = [np.loadtxt('./data/EnKF/state_%02d.dat' % (i+1)) for i in range(n_ensemble)]
v_true = np.loadtxt('./data/state_true.dat')

In [ ]:
it = 60

fig, ax = plt.subplots(figsize=(8,3))
for i in range(n_ensemble):
    label = 'simulation (free run)' if i == 0 else '__nolegend__'
    plt.plot(x, v_sim[i][it,:], '-', c='0.6', lw=0.5, label=label)
plt.plot(ad.x, v_true[it,:], '-', c='C3', lw=2.0, label='true')
plt.plot(ad.x_obs, v_obs[it,:], '^', c='C0', ms=9, label='observation')
plt.xlim(0, 400)
plt.ylim(-40, 40)
plt.xlabel(r'$x$ [km]')
plt.ylabel(r'$C$')
plt.legend(loc='lower right', framealpha=0.7)
plt.grid(ls='--', c='0.5', lw=0.3)
plt.title(r'EnKF ($t={:4.1f}$)'.format(it * 7.5))
plt.tight_layout()